In [ ]:
import os
import pandas as pd

data_path = "/mnt/L-HDD/dataset/splitted.csv"
df = pd.read_csv(data_path)

In [6]:
df

,unique_id,id,time,type,x,y,hit_sound,repeat,length,spinner_time,...,path_75,path_76,path_77,path_78,path_79,path_80,path_81,path_82,path_83,path_84
0,6537,1178020-0,130708.000000,circle,234,84,2,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6886,1178020-0,165468.000000,slider,361,252,0,1,44.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6885,1178020-0,165324.000000,slider,466,373,0,1,44.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6884,1178020-0,165035.000000,circle,366,50,2,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6883,1178020-0,164963.000000,circle,307,97,2,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28788,13188,2333780-4,62441.000000,circle,320,212,0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28789,13187,2333780-4,61879.000000,slider,188,300,0,1,120.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28790,13186,2333780-4,61504.000000,circle,76,344,0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28791,13194,2333780-4,64504.000000,circle,416,324,2,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df[df["path_79"].notna()]

,unique_id,id,time,type,x,y,hit_sound,repeat,length,spinner_time,...,path_75,path_76,path_77,path_78,path_79,path_80,path_81,path_82,path_83,path_84


In [9]:
df.columns[:40]

Index(['unique_id', 'id', 'time', 'type', 'x', 'y', 'hit_sound', 'repeat',
       'length', 'spinner_time', 'beatmap_id', 'rms', 'beat_length', 'meter',
       'slider_velocity', 'sample_set', 'volume', 'effects', 'mfcc_1',
       'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8',
       'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13', 'mfcc_14',
       'mfcc_15', 'mfcc_16', 'mfcc_17', 'mfcc_18', 'mfcc_19', 'mfcc_20',
       'curve_type', 'path_1'],
      dtype='object')

In [13]:
categorical_columns = ["type", "hit_sound", "sample_set", "effects"]
df = pd.get_dummies(df, columns=categorical_columns, prefix=categorical_columns)

In [25]:
df["path_1"].max()

np.float64(534.0)

In [32]:
df[df["path_2"] > 400]

,unique_id,id,time,x,y,repeat,length,spinner_time,beatmap_id,rms,...,hit_sound_8,hit_sound_10,hit_sound_12,hit_sound_14,sample_set_0.0,sample_set_1.0,sample_set_2.0,sample_set_3.0,effects_0.0,effects_1.0
1363,6152,1178020-0,54554.0,115,320,1,478.500015,0,1178020,0.254671,...,False,False,False,False,False,False,True,False,True,False
7857,9495,1178020-5,49939.0,96,368,1,300.000000,0,1178020,0.279549,...,False,False,False,False,False,False,True,False,True,False
8200,9773,1178020-5,108785.0,0,334,2,50.000000,0,1178020,0.318862,...,False,False,False,False,False,False,True,False,True,False
8201,9772,1178020-5,108497.0,74,319,2,50.000000,0,1178020,0.154920,...,False,False,False,False,False,False,True,False,True,False


In [17]:
df.columns[df.columns.str.startswith("path_")]

Index(['path_1', 'path_2', 'path_3', 'path_4', 'path_5', 'path_6', 'path_7',
       'path_8', 'path_9', 'path_10',
       ...
       'path_7320', 'path_7321', 'path_7322', 'path_7323', 'path_7324',
       'path_7325', 'path_7326', 'path_7327', 'path_7328', 'path_7329'],
      dtype='object', length=7330)

In [8]:
df["Path"] = df["Path"].fillna("E")

In [9]:
df.to_csv("/mnt/L-HDD/dataset/hit_objects_formatted_new.csv", index=False)

In [13]:
df["RMS"]

0         [0.11176227]
1         [0.26603892]
2         [0.24931613]
3          [0.2306636]
4         [0.20984024]
              ...     
532700    [0.33295044]
532701    [0.37319627]
532702    [0.37515637]
532703    [0.33485928]
532704    [0.33707836]
Name: RMS, Length: 532705, dtype: string

In [4]:
a = df.fillna('0')

KeyboardInterrupt: 

In [13]:
a

,unique_id,ID,Time,Type,X,Y,HitSound,Path,Repeat,Length,beatmap_id,MFCC,RMS,beat_length,meter,slider_velocity,sample_set,volume,effects
0,1,2331782-0,161,circle,381,112,0,0,0,0.000000,2331782,[-527.34375 0. 0. 0. ...,0.002005,337.078652,4.0,-180.000000,1.0,60.0,0.0
1,2,2331782-0,498,circle,392,102,0,0,0,0.000000,2331782,[-468.2952 19.879934 -40.841194 -49.7...,0.028299,337.078652,4.0,-180.000000,1.0,60.0,0.0
2,3,2331782-0,835,circle,406,96,0,0,0,0.000000,2331782,[-485.77444 16.343855 -23.927628 -31.3...,0.029729,337.078652,4.0,-180.000000,1.0,60.0,0.0
3,4,2331782-0,1173,slider,420,94,0,B|285:83,1,135.000005,2331782,[-325.0762 92.5908 -46.830757 -43.7...,0.042683,337.078652,4.0,-133.333333,2.0,25.0,0.0
4,5,2331782-0,1678,slider,216,16,0,P|186:76|202:140,1,135.000005,2331782,[-347.58185 115.06575 -55.66121 -62.8...,0.072165,337.078652,4.0,-133.333333,2.0,25.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,706739-0,6521,slider,420,255,0,L|342:246,1,56.000000,706739,[-3.5148135e+02 1.9661513e+02 4.8828384e+01 ...,0.101223,652.173913,4.0,-112.000000,1.0,60.0,0.0
49996,49997,706739-0,7010,slider,335,275,0,L|279:281,1,56.000000,706739,[-362.36908 190.58273 66.44863 -30.8733...,0.088089,652.173913,4.0,-112.000000,1.0,60.0,0.0
49997,49998,706739-0,7499,circle,246,260,2,0,0,0.000000,706739,[-373.52493 188.651 64.729164 -27.4...,0.100122,652.173913,4.0,-112.000000,1.0,60.0,0.0
49998,49999,706739-0,7826,slider,179,302,0,P|165:266|168:235,1,56.000000,706739,[-3.4629617e+02 1.8040866e+02 2.8912586e+01 ...,0.094520,652.173913,4.0,-112.000000,1.0,60.0,0.0


In [14]:
res = True
for i in range(len(old_df)):
    res = res and (old_df.iloc[i] == df.iloc[i]).any()
res

np.True_

In [ ]:
categorical_cols = ["Type", "HitSound", "sample_set", "effects"]

In [ ]:
# osu grid -> 512x384

In [3]:
# It's about nobarline. When nobarline is on, it flips the 4. bit which means adding 8.

def correct_effect_value(x):
    if(x > 8):
        return x != 5746
    return 0 if x == 8 else x

df["effects"] = df["effects"].apply(correct_effect_value)

In [3]:
categorical_cols = ["Type", "HitSound", "sample_set", "effects"]
df = pd.get_dummies(df, columns=categorical_cols)

NameError: name 'pd' is not defined

In [4]:
from sklearn.preprocessing import MinMaxScaler

df["RMS"] = df["RMS"].str.replace("[", "").str.replace("]", "")
numerical_cols = ["X", "Y", "beat_length", "meter", "slider_velocity", "volume", "RMS"]
scaler = MinMaxScaler()

df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

NameError: name 'df' is not defined